# Constant-time extended GCD

## Outline :

1. SageMath System and GCD

2. DivStep and constant-time GCD

3. JumpDivStep and constant-time GCD


## Section 1: SageMath System and GCD

### SageMath: http://www.sagemath.org/



In [1]:
#The enviroments:

p = 761; q61 = 765; q = 6*q61+1; w = 250
Zx.<x> = ZZ[]; R.<xp> = Zx.quotient(x^p-x-1)
Fq = GF(q); Fqx.<xq> = Fq[]; Rq.<xqp> = Fqx.quotient(x^p-x-1)
F3 = GF(3); F3x.<x3> = F3[]; R3.<x3p> = F3x.quotient(x^p-x-1)

In [2]:

# functions for lists

def conditional_swap( condition , a , b ) :
  if 0 == condition:
    return a , b
  else :
    return b , a


def list_shift(l,shift) :
  src_idx = max(-shift,0)
  dst_idx = max(shift,0)
  length = max(len(l)-abs(shift),0)
  new_l = [ l[0].parent()(0) ]* len(l)
  new_l[dst_idx:dst_idx+length] = l[src_idx:src_idx+length]
  return new_l


def list_pad_zero( al , len ):
  al_ref = [ al[0].parent(0) ]*(len)
  al = map( lambda x,y: y if x is None else x, al , al_ref )
  return al


def list_pad_front(al,nlen):
  assert nlen >= len(al)
  ral = list(al)
  ral[:0] = [al[0].parent(0)]*(nlen-len(al))
  return ral


def list_mul( l , c ):
  return [ x*c for x in l ]

def list_add( l1 , l2 ):
  return [ x+y for x,y in zip(l1,l2) ]


###################################

# functions for polynomials(lists)

def poly_in_deg( a , deg ):
  return list_pad_zero( a.list() , deg+1 )


def poly_rev( a , deg ):
  r = poly_in_deg(a,deg)
  rev_r = r[::-1]
  return rev_r


def poly_mul( a , b ):
  r = [a[0].parent(0)]*(len(a)+len(b)-1)
  for i in range(len(a)):
    r[i:i+len(b)] = list_add( r[i:i+len(b)] , list_mul(b,a[i]) )
  return r


###################################
##
##  This is not for a normal matrix. 
##  It is for the "transition matrix".
##
###################################

def mat_mul_vec( mat , vec ) :
  m_l = len(mat[0][0])
  f_m00 = poly_mul(mat[0][0],vec[0])
  g_m01 = poly_mul(mat[0][1],vec[1])
  f = list_add( f_m00 , g_m01 )
  f_m10 = poly_mul(mat[1][0],vec[0])
  g_m11 = poly_mul(mat[1][1],vec[1])
  g = list_add( f_m10 , g_m11 )
  ## XXX: assert 0 for truncated part.
  rf = f[m_l-1:]
  rg = list_pad_zero( g[m_l:] , len(vec[1]) )
  return  rf , rg 


def mat_mul( mat1 , mat2 ) :
  m1_len = len(mat1[0][0])
  m2_len = len(mat2[0][0])
  r_len = m1_len+m2_len
  r00_u = poly_mul(mat1[0][0],mat2[0][0])
  r00_d = poly_mul(mat1[0][1],mat2[1][0])
  r00 = list_add(list_pad_front(r00_u,r_len),list_pad_zero(r00_d,r_len))
  r01_u = poly_mul(mat1[0][0],mat2[0][1])
  r01_d = poly_mul(mat1[0][1],mat2[1][1])
  r01 = list_add(list_pad_front(r01_u,r_len),list_pad_zero(r01_d,r_len))
  r10_u = poly_mul(mat1[1][0],mat2[0][0])
  r10_d = poly_mul(mat1[1][1],mat2[1][0])
  r10 = list_add(list_pad_front(r10_u,r_len),list_pad_zero(r10_d,r_len))
  r11_u = poly_mul(mat1[1][0],mat2[0][1])
  r11_d = poly_mul(mat1[1][1],mat2[1][1])
  r11 = list_add(list_pad_front(r11_u,r_len),list_pad_zero(r11_d,r_len))
  return [ [r00,r01] , [ r10 , r11 ] ]

def deg1mat_mul( deg1mat , mat2 ) :
  inp_poly_len = len(mat2[0][0])
  np_len = inp_poly_len+1
  mat2_r0 = [list_pad_front(mat2[0][0],np_len),list_pad_front(mat2[0][1],np_len)]
  mat2_r1 = [list_pad_zero(mat2[1][0],np_len),list_pad_zero(mat2[1][1],np_len)]
  r0 , r0_ = conditional_swap( deg1mat[0][0][0] , mat2_r1 , mat2_r0 )   # use AND
  m1_10 = deg1mat[1][0][0]
  m1_11 = deg1mat[1][1][0]
  r10_u = list_mul(mat2[0][0],m1_10)
  r10_d = list_mul(mat2[1][0],m1_11)
  r11_u = list_mul(mat2[0][1],m1_10)
  r11_d = list_mul(mat2[1][1],m1_11)
  r10 = list_add(list_pad_front(r10_u,np_len),list_pad_zero(r10_d,np_len))
  r11 = list_add(list_pad_front(r11_u,np_len),list_pad_zero(r11_d,np_len))
  return [ r0 , [ r10 , r11 ] ]


In [3]:
# Some variables for testing

def random_poly( deg ):
  r = Fqx.random_element()
  r = Fqx(r-r + Fq.random_element())
  for i in range(deg):
    r = Fqx( r*xq + Fq.random_element() )
  return r

a1 = random_poly(1)
b1 = random_poly(1)
c1 = random_poly(1)
a2 = random_poly(2)
b2 = random_poly(2)
c2 = random_poly(2)
a3 = random_poly(3)
b3 = random_poly(3)
c3 = random_poly(3)

In [4]:
print a1,b1,c1
print a2,b2,c2
print a3,b3,c3

2311*xq + 273 3558*xq + 851 4214*xq + 1311
580*xq^2 + 2639*xq + 2921 4303*xq^2 + 2617*xq + 3932 1001*xq^2 + 2272*xq + 3672
4221*xq^3 + 945*xq^2 + 1411*xq + 3036 2897*xq^3 + 3580*xq^2 + 882*xq + 4063 2179*xq^3 + 1970*xq^2 + 2408*xq + 3868


In [36]:
# The standard GCD in the textbook

#output: gcd = s*f + t*g
def ext_gcd( f , g ) :
  if f.degree() < g.degree() :  f,g = g,f  #swap
  T = f.parent()
  lr = [ f , g ]
  ls = [ T(1) , T(0) ]
  lt = [ T(0) , T(1) ]
  #print "lr: [f,g] = ", lr,'\nls:',ls, "\nlt:", lt, '\n'
  print "lr[0]: " , lr[0] , "\tls[0]:", ls[0] , "\tlt[0]:", lt[0] 
  print "lr[1]: " , lr[1] , "\tls[1]:", ls[1] , "\tlt[0]:", lt[1] 
  print ""
  while lr[-1] != 0 :
    qq = lr[-2] // lr[-1]
    rr = lr[-2] % lr[-1]
    lr.append( rr )
    ls.append( ls[-2] - qq*ls[-1] )
    lt.append( lt[-2] - qq*lt[-1] )
    print "lr[-1]:",lr[-1] , "\tls[-1]:", ls[-1], "\tlt[-1]" , lt[-1]
  print "\nlr:", lr,'\nls:',ls, "\nlt:", lt, '\n'
  #normalize
  #inv_n = (lr[-2].coefficients()[0])^-1
  inv_n = lr[-2].leading_coefficient()^-1
  return lr[-2]*inv_n,ls[-2]*inv_n,lt[-2]*inv_n

ext_gcd( a2, b3 )

lr[0]:  2897*xq^3 + 3580*xq^2 + 882*xq + 4063 	ls[0]: 1 	lt[0]: 0
lr[1]:  580*xq^2 + 2639*xq + 2921 	ls[1]: 0 	lt[0]: 1

lr[-1]: xq + 3564 	ls[-1]: 1 	lt[-1] 2900*xq + 1680
lr[-1]: 1145 	ls[-1]: 4011*xq + 3122 	lt[-1] 2897*xq^2 + 3831*xq + 2039
lr[-1]: 0 	ls[-1]: 3128*xq^2 + 3214*xq + 1442 	lt[-1] 3975*xq^3 + 3806*xq^2 + 4077*xq + 4399

lr: [2897*xq^3 + 3580*xq^2 + 882*xq + 4063, 580*xq^2 + 2639*xq + 2921, xq + 3564, 1145, 0] 
ls: [1, 0, 1, 4011*xq + 3122, 3128*xq^2 + 3214*xq + 1442] 
lt: [0, 1, 2900*xq + 1680, 2897*xq^2 + 3831*xq + 2039, 3975*xq^3 + 3806*xq^2 + 4077*xq + 4399] 



(1, 1463*xq + 2621, 616*xq^2 + 4450*xq + 928)

In [6]:
#first variant: Performing operations as matrix-multiplication of 2x2 matrices.

def ext_gcd1( f , g ) :
  if f.degree() < g.degree() :  f,g = g,f  #swap
  Ta = f.parent()
  x = Ta.gen()
  lr = [ f , g ]
  ls = [Ta(1),Ta(0)]
  lt = [Ta(0),Ta(1)]
  print "[f,g]:" , lr,"\n",ls,"\n",lt , "\n"
  while lr[-1] != 0 :
    if lr[0].degree() > lr[1].degree() : 
       lr = lr[::-1]
       ls , lt = lt , ls
       print "op0: [0,1] <-- swap, choose g"
    else:
       print "op0: [1,0] <-- choose f"
    delta = lr[1].degree() - lr[0].degree()
    lc_f = lr[0].leading_coefficient()
    lc_g = lr[1].leading_coefficient()
    lr[1] = lr[1]*lc_f - lr[0]*lc_g*(x^delta)
    lt[0] = lt[0]*lc_f-ls[0]*lc_g*(x^delta)
    lt[1] = lt[1]*lc_f-ls[1]*lc_g*(x^delta)
    print "op1: [" , lc_f , "," , lc_g , "x^" , delta , "]"
    print "-->\n[f,g]:" , lr,"\n",ls,"\n",lt , "\n"
  #normalize
  inv_n = (lr[-2].leading_coefficient())^-1
  return lr[-2]*inv_n,map(lambda x:x*inv_n, ls )

# demo code:
ext_gcd1( a2, b2 )

[f,g]: [580*xq^2 + 2639*xq + 2921, 4303*xq^2 + 2617*xq + 3932] 
[1, 0] 
[0, 1] 

op0: [1,0] <-- choose f
op1: [ 580 , 4303 x^ 0 ]
-->
[f,g]: [580*xq^2 + 2639*xq + 2921, 756*xq + 4519] 
[1, 0] 
[288, 580] 

op0: [0,1] <-- swap, choose g
op1: [ 756 , 580 x^ 1 ]
-->
[f,g]: [756*xq + 4519, 3031*xq + 5] 
[288, 580] 
[2827*xq + 756, 3334*xq] 

op0: [1,0] <-- choose f
op1: [ 756 , 3031 x^ 0 ]
-->
[f,g]: [756*xq + 4519, 1644] 
[288, 580] 
[2397*xq + 1614, 45*xq + 373] 

op0: [0,1] <-- swap, choose g
op1: [ 1644 , 756 x^ 1 ]
-->
[f,g]: [1644, 998] 
[2397*xq + 1614, 45*xq + 373] 
[1313*xq^2 + 1022*xq + 599, 2708*xq^2 + 2654*xq + 3183] 

op0: [1,0] <-- choose f
op1: [ 1644 , 998 x^ 0 ]
-->
[f,g]: [1644, 0] 
[2397*xq + 1614, 45*xq + 373] 
[802*xq^2 + 4158*xq + 2951, 3273*xq^2 + 2726*xq + 3320] 



(1, [2808*xq + 822, 1064*xq + 2494])

## Section 2: DivStep and constant-time GCD

In [7]:

# One 'division step' in the GCD.
# Trying to make it time-constant by applying 'conditional swap'

def divstep( delta , f , g ):
  coef_k = f[0].parent()
  kx.<x> = coef_k[] #kx = T_[0][0].parent()
  #x = kx.gen()
  #M2kx = T_.parent() 
  M2kx = MatrixSpace(kx.fraction_field(),2)   
  f = list_pad_zero( f , max(len(f),len(g)))
  g = list_pad_zero( g , max(len(f),len(g)))
  T0 = M2kx( (1,0,-g[0]/x,f[0]/x) )
  T1 = M2kx( (0,1,g[0]/x,-f[0]/x) )
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rc , r1 = conditional_swap( c , [delta,f,g] , [-delta,g,f] )
  T0 , T1 = conditional_swap( c , T0 , T1 )
  # elimination
  rc[0] = 1 + rc[0]
  f = rc[1]
  g = rc[2]
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  #rg.pop(0)
  rg = list_shift(rg,-1)
  return rc[0] , rc[1] , rg , T0

# demo code:
# Warning: The lists are in the reverse order of polynomials
#
f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
print "[f,g] = ", [ f, g ] , "\ndivstep(0,f,g): -->"
divstep( 0 , f , g )



[f,g] =  [[580, 2639, 2921], [4303, 2617, 3932]] 
divstep(0,f,g): -->


(
                                      [    1     0]
1, [580, 2639, 2921], [756, 4519, 0], [288/x 580/x]
)

In [8]:

# The reversed Constant-time GCD

def ext_gcd2( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  #if deg_a == deg_b : 
  #  b = b*a.leading_coefficient() - a*b.leading_coefficient()
  #deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  M2kx = MatrixSpace( a.parent().fraction_field() , 2 )
  T0 = M2kx(1)
  print delta , r1 , r2 , "\n", T0 , "\n"
  delta , r1 , r2 , T1 = divstep( delta , r1 , r2 )
  T0 = M2kx(T1)*T0
  print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep( delta , r1 , r2 )
    T0 = M2kx(T1)*T0
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0] = T0[0]*inv
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0

# demo code:
ext_gcd2( a2, b2 )


0 [580, 2639, 2921] [4303, 2617, 3932] 
[1 0]
[0 1] 

1 [580, 2639, 2921] [756, 4519, 0] 
[    1     0]
[288/x 580/x] 
[     1      0]
[288/xq 580/xq] 

0 [756, 4519, 0] [3031, 5, 0] 
[     0      1]
[ 756/x 4011/x] 
[              288/xq               580/xq]
[(756*xq + 2827)/xq^2            3334/xq^2] 

1 [756, 4519, 0] [1644, 0, 0] 
[     1      0]
[1560/x  756/x] 
[               288/xq                580/xq]
[(1614*xq + 2397)/xq^3    (373*xq + 45)/xq^3] 

0 [1644, 0, 0] [998, 0, 0] 
[     0      1]
[1644/x 3835/x] 
[            (1614*xq + 2397)/xq^3                (373*xq + 45)/xq^3]
[ (599*xq^2 + 1022*xq + 1313)/xq^4 (3183*xq^2 + 2654*xq + 2708)/xq^4] 

output:
0 [1, 0, 0] [998, 0, 0] 
[             (822*xq + 2808)/xq^3             (2494*xq + 1064)/xq^3]
[ (599*xq^2 + 1022*xq + 1313)/xq^4 (3183*xq^2 + 2654*xq + 2708)/xq^4] 



(
[1, 0, 0],

[             (822*xq + 2808)/xq^3             (2494*xq + 1064)/xq^3]
[ (599*xq^2 + 1022*xq + 1313)/xq^4 (3183*xq^2 + 2654*xq + 2708)/xq^4]
)

In [9]:
ext_gcd2( a2*b1 , a1*b1 )

1 [2281, 3310, 4275, 2040] [57, 4346, 2773] 
[1 0]
[0 1] 

0 [57, 4346, 2773, 0] [3773, 1537, 1505, 0] 
[     0      1]
[  57/x 2310/x] 
[      0       1]
[  57/xq 2310/xq] 

1 [57, 4346, 2773, 0] [1974, 3507, 0, 0] 
[    1     0]
[818/x  57/x] 
[                   0                    1]
[           3249/xq^2 (818*xq + 3122)/xq^2] 

0 [1974, 3507, 0, 0] [530, 1430, 0, 0] 
[     0      1]
[1974/x 4534/x] 
[                        3249/xq^2              (818*xq + 3122)/xq^2]
[                        3038/xq^3 (1974*xq^2 + 3875*xq + 1095)/xq^3] 

1 [1974, 3507, 0, 0] [0, 0, 0, 0] 
[     1      0]
[4061/x 1974/x] 
[                        3249/xq^2              (818*xq + 3122)/xq^2]
[            (4246*xq + 1166)/xq^4 (1522*xq^2 + 3335*xq + 3760)/xq^4] 

2 [1974, 3507, 0, 0] [0, 0, 0, 0] 
[     1      0]
[     0 1974/x] 
[                        3249/xq^2              (818*xq + 3122)/xq^2]
[            (3029*xq + 1593)/xq^5 (1914*xq^2 + 4387*xq + 3184)/xq^5] 

output:
2 [1, 2688, 0, 0] [0,

(
[1, 2688, 0, 0],

[                        3260/xq^2             (3596*xq + 2118)/xq^2]
[            (3029*xq + 1593)/xq^5 (1914*xq^2 + 4387*xq + 3184)/xq^5]
)

In [10]:
ext_gcd1( a2*b1 , a1*b1 )

[f,g]: [2281*xq^3 + 3310*xq^2 + 4275*xq + 2040, 57*xq^2 + 4346*xq + 2773] 
[1, 0] 
[0, 1] 

op0: [0,1] <-- swap, choose g
op1: [ 57 , 2281 x^ 1 ]
-->
[f,g]: [57*xq^2 + 4346*xq + 2773, 3773*xq^2 + 1537*xq + 1505] 
[0, 1] 
[57, 2310*xq] 

op0: [1,0] <-- choose f
op1: [ 57 , 3773 x^ 0 ]
-->
[f,g]: [57*xq^2 + 4346*xq + 2773, 1974*xq + 3507] 
[0, 1] 
[3249, 3122*xq + 818] 

op0: [0,1] <-- swap, choose g
op1: [ 1974 , 57 x^ 1 ]
-->
[f,g]: [1974*xq + 3507, 530*xq + 1430] 
[3249, 3122*xq + 818] 
[3038*xq, 1095*xq^2 + 3875*xq + 1974] 

op0: [1,0] <-- choose f
op1: [ 1974 , 530 x^ 0 ]
-->
[f,g]: [1974*xq + 3507, 0] 
[3249, 3122*xq + 818] 
[1166*xq + 4246, 3760*xq^2 + 3335*xq + 1522] 



(xq + 2688, [3260, 2118*xq + 3596])

In [11]:
# The "list" version of previous ext_ext2()

def divstep_list( delta , f , g ):
  coef_k = f[0].parent()
  T00 = [ [coef_k(1)] , [coef_k(0)] ]
  T10 = [ [-g[0]] , [f[0]]  ]
  T01 = [ [coef_k(0)] , [coef_k(1)] ]
  T11 = [ [g[0]]  , [-f[0]] ]
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rd , rd_ = conditional_swap( c , delta , -delta )
  f  , g   = conditional_swap( c , f , g )
  T0 , T0_ = conditional_swap( c , [ T00, T10 ] , [ T01 , T11 ] )
  # elimination
  rd = rd + 1
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  rg = list_shift( rg , -1 )
  return rd , f , rg , T0

f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
print "[f,g] = ", [ f, g ] , "\ndivstep(0,f,g): -->"
divstep_list( 0 , f , g )


[f,g] =  [[580, 2639, 2921], [4303, 2617, 3932]] 
divstep(0,f,g): -->


(1, [580, 2639, 2921], [756, 4519, 0], [[[1], [0]], [[288], [580]]])

In [12]:

def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  #if deg_a == deg_b : 
  #  b = b*a.leading_coefficient() - a*b.leading_coefficient()
  #deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  coef_k = r1[0].parent()
  print delta , r1 , r2 , "\n"
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = T1    # T1*Identity = T1
  print delta , r1 , r2 , '\n' , T1 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = deg1mat_mul( T1 , T0 )
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = list_mul( T0[0][0] , inv )
  T0[0][1] = list_mul( T0[0][1] , inv )
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0


# demo code:
#extgcd_list( a2 , b2 )
extgcd_list( a2*b1 , a1*b1 )



1 [2281, 3310, 4275, 2040] [57, 4346, 2773, 0] 

0 [57, 4346, 2773, 0] [3773, 1537, 1505, 0] 
[[[0], [1]], [[57], [2310]]] 

1 [57, 4346, 2773, 0] [1974, 3507, 0, 0] 
[[[1], [0]], [[818], [57]]] 
[[[0, 0], [0, 1]], [[3249, 0], [3122, 818]]] 

0 [1974, 3507, 0, 0] [530, 1430, 0, 0] 
[[[0], [1]], [[1974], [4534]]] 
[[[3249, 0, 0], [3122, 818, 0]], [[3038, 0, 0], [1095, 3875, 1974]]] 

1 [1974, 3507, 0, 0] [0, 0, 0, 0] 
[[[1], [0]], [[4061], [1974]]] 
[[[0, 3249, 0, 0], [0, 3122, 818, 0]], [[1166, 4246, 0, 0], [3760, 3335, 1522, 0]]] 

2 [1974, 3507, 0, 0] [0, 0, 0, 0] 
[[[1], [0]], [[0], [1974]]] 
[[[0, 0, 3249, 0, 0], [0, 0, 3122, 818, 0]], [[1593, 3029, 0, 0, 0], [3184, 4387, 1914, 0, 0]]] 

output:
2 [1, 2688, 0, 0] [0, 0, 0, 0] 
[[[0, 0, 3260, 0, 0], [0, 0, 2118, 3596, 0]], [[1593, 3029, 0, 0, 0], [3184, 4387, 1914, 0, 0]]] 



([1, 2688, 0, 0],
 [[[0, 0, 3260, 0, 0], [0, 0, 2118, 3596, 0]],
  [[1593, 3029, 0, 0, 0], [3184, 4387, 1914, 0, 0]]])

## Section 3: JumpDivStep and constant-time GCD

In [13]:


def jumpdivsteps(n,t,delta,f,g):
  print "jumpdivsteps(",n,t,delta,f,g,")"
  assert t>= n and n>=0
  f,g = f.truncate(t), g.truncate(t)
  print "truncated: f=",f , "  g=", g
  kx=f.parent()
  x=kx.gen()
  M2kx=MatrixSpace(kx.fraction_field(),2)
  if n == 0: return delta,f,g,(),M2kx(1)
  if n==1:
    c = 1 if delta > 0 and g[0] != 0 else 0
    Tc, Td = conditional_swap( c , M2kx((1,0,-g[0]/x,f[0]/x)), M2kx( (0,1,g[0]/x,-f[0]/x) ) )
    delta , d2 = conditional_swap( c , delta , -delta )
    fc , f2 = conditional_swap( c , f , g )
    gc , g2 = conditional_swap( c , f[0]*g-g[0]*f , g[0]*f-f[0]*g )
    return 1+delta,fc,kx(gc/x),(Tc,),Tc
  # other cases of n
  j = n//2
  print "j=",j
  delta,f1,g1,T1,P1 = jumpdivsteps(j,j,delta,f,g)
  print "P1:\n" , P1 
  f,g = P1*vector( (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-j),kx(g).truncate(t-j)
  print "truncated: f=",f , "  g=", g
  delta,f2,g2,T2,P2 = jumpdivsteps(n-j,n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = P2*vector( (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated: f=",f , "  g=", g
  return delta,f,g,T2+T1,P2*P1

# demo code:
f = a2.reverse( 2 )
g = b2.reverse( 2 )
delta, f , g , T2 , T1 = jumpdivsteps( 4 , 5 , 0 , f , g )
print "\ndelta:" , delta, "[f,g] = " , [f,g]
print T2
print T1


jumpdivsteps( 4 5 0 2921*xq^2 + 2639*xq + 580 3932*xq^2 + 2617*xq + 4303 )
truncated: f= 2921*xq^2 + 2639*xq + 580   g= 3932*xq^2 + 2617*xq + 4303
j= 2
jumpdivsteps( 2 2 0 2921*xq^2 + 2639*xq + 580 3932*xq^2 + 2617*xq + 4303 )
truncated: f= 2639*xq + 580   g= 2617*xq + 4303
j= 1
jumpdivsteps( 1 1 0 2639*xq + 580 2617*xq + 4303 )
truncated: f= 580   g= 4303
P1:
[     1      0]
[288/xq 580/xq]
P1x(f,g): f= 2639*xq + 580   g= 756
truncated: f= 580   g= 756
jumpdivsteps( 1 1 1 580 756 )
truncated: f= 580   g= 756
P2:
[      0       1]
[ 756/xq 4011/xq]
P2x(f,g): f= 756   g= 0
truncated: f= 756   g= 0
P1:
[              288/xq               580/xq]
[(756*xq + 2827)/xq^2            3334/xq^2]
P1x(f,g): f= 4519*xq + 756   g= 5*xq + 3031
truncated: f= 4519*xq + 756   g= 5*xq + 3031
jumpdivsteps( 2 2 0 4519*xq + 756 5*xq + 3031 )
truncated: f= 4519*xq + 756   g= 5*xq + 3031
j= 1
jumpdivsteps( 1 1 0 4519*xq + 756 5*xq + 3031 )
truncated: f= 756   g= 3031
P1:
[      1       0]
[1560/xq  756/xq]
P

In [14]:
def jumpgcd(R0,R1):
  d = R0.degree()
  assert d>0 and d>R1.degree()
  f,g = R0.reverse(d), R1.reverse(d-1)
  delta,f,g,T,P = jumpdivsteps(2*d-1,3*d-1,1,f,g)
  return f.reverse(delta//2)/f[0]

# demo code:
f = a2.reverse( 2 )
g = b1.reverse( 1 )
jumpgcd(f,g)


jumpdivsteps( 3 5 1 580*xq^2 + 2639*xq + 2921 3558*xq + 851 )
truncated: f= 580*xq^2 + 2639*xq + 2921   g= 3558*xq + 851
j= 1
jumpdivsteps( 1 1 1 580*xq^2 + 2639*xq + 2921 3558*xq + 851 )
truncated: f= 2921   g= 851
P1:
[      0       1]
[ 851/xq 1670/xq]
P1x(f,g): f= 3558*xq + 851   g= 2343*xq + 1896
truncated: f= 3558*xq + 851   g= 2343*xq + 1896
jumpdivsteps( 2 2 0 3558*xq + 851 2343*xq + 1896 )
truncated: f= 3558*xq + 851   g= 2343*xq + 1896
j= 1
jumpdivsteps( 1 1 0 3558*xq + 851 2343*xq + 1896 )
truncated: f= 851   g= 1896
P1:
[      1       0]
[2695/xq  851/xq]
P1x(f,g): f= 3558*xq + 851   g= 4201
truncated: f= 851   g= 4201
jumpdivsteps( 1 1 1 851 4201 )
truncated: f= 851   g= 4201
P2:
[      0       1]
[4201/xq 3740/xq]
P2x(f,g): f= 4201   g= 0
truncated: f= 4201   g= 0
P2:
[              2695/xq                851/xq]
[(4201*xq + 2055)/xq^2             1177/xq^2]
P2x(f,g): f= 4201   g= 3453
truncated: f= 4201   g= 3453


1

In [15]:

# The list version of previous JumpGCD

def jumpdivstep_mat(n,delta,f,g):
  print "jumpdivstep_mat(",n,delta,f,g,")"
  assert n>0
  ### truncate 
  f,g = f[:n], g[:n]
  print "truncated(",n,"): f=",f , "  g=", g
  if n==1:
    rd,f,g,T = divstep_list( delta , f , g )
    return rd,T
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  #f,g = mat_mul_vec( P2 , (f,g) )
  #print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  #print "truncated: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  print "ret: ", delta , rP
  return delta, rP

def jumpdivsteps_list(n,delta,f,g):
  print "jumpdivsteps_list(",n,delta,f,g,")"
  assert n>0
  if n==1:
    return divstep_list( delta , f , g )
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = mat_mul_vec( P2 , (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated ???: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  return delta, f , g , rP


# demo code:
f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
jumpdivsteps_list( 5 , 0 , f , g )


jumpdivsteps_list( 5 0 [580, 2639, 2921] [4303, 2617, 3932] )
j= 2
jumpdivstep_mat( 2 0 [580, 2639, 2921] [4303, 2617, 3932] )
truncated( 2 ): f= [580, 2639]   g= [4303, 2617]
j= 1
jumpdivstep_mat( 1 0 [580, 2639] [4303, 2617] )
truncated( 1 ): f= [580]   g= [4303]
P1:
[[[1], [0]], [[288], [580]]]
P1x(f,g): f= [580, 2639]   g= [756, 0]
jumpdivstep_mat( 1 1 [580, 2639] [756, 0] )
truncated( 1 ): f= [580]   g= [756]
P2:
[[[0], [1]], [[756], [4011]]]
ret:  0 [[[288, 0], [580, 0]], [[2827, 756], [3334, 0]]]
P1:
[[[288, 0], [580, 0]], [[2827, 756], [3334, 0]]]
P1x(f,g): f= [756, 4519, 0]   g= [3031, 5, 0]
jumpdivstep_mat( 3 0 [756, 4519, 0] [3031, 5, 0] )
truncated( 3 ): f= [756, 4519, 0]   g= [3031, 5, 0]
j= 1
jumpdivstep_mat( 1 0 [756, 4519, 0] [3031, 5, 0] )
truncated( 1 ): f= [756]   g= [3031]
P1:
[[[1], [0]], [[1560], [756]]]
P1x(f,g): f= [756, 4519, 0]   g= [1644, 0, 0]
jumpdivstep_mat( 2 1 [756, 4519, 0] [1644, 0, 0] )
truncated( 2 ): f= [756, 4519]   g= [1644, 0]
j= 1
jumpdivstep_ma

(1,
 [1644, 0, 0],
 [0, 0, 0],
 [[[0, 2397, 1614, 0, 0], [0, 45, 373, 0, 0]],
  [[802, 4158, 2951, 0, 0], [3273, 2726, 3320, 0, 0]]])

In [55]:

def jumpgcd_list(R0,R1):
  d = R0.degree()
  d1 = R1.degree()
  assert d>0 and d>d1
  f = poly_rev(R0,d)
  g = poly_rev(R1,d1)
  g = list_pad_zero(g,len(f))
  delta,f,g,T = jumpdivsteps_list(2*d-1,1,f,g)
  #return delta,f,g,T
  gcd_deg = delta>>1
  gcd_len = gcd_deg + 1
  inv = f[0]^-1
  r_gcd = list_mul( f , inv )
  r_s = list_mul( T[0][0] , inv )
  r_t = list_mul( T[0][1] , inv )
  deg_s = d1 - gcd_deg -1
  deg_t = d - gcd_deg -1
  return gcd_len , r_gcd , r_s[gcd_deg*2:gcd_deg*2+deg_s+1] , r_t[gcd_deg*2:gcd_deg*2+deg_t+1]

ff, gg = a1*b1*c1 , b2
ext_gcd( ff , gg  )


lr[0]:  1466*xq^3 + 1816*xq^2 + 1502*xq + 3922 	ls[0]: 1 	lt[0]: 0
lr[1]:  4303*xq^2 + 2617*xq + 3932 	ls[1]: 0 	lt[0]: 1

lr[-1]: 138*xq + 3916 	ls[-1]: 1 	lt[-1] 2460*xq + 1254
lr[-1]: 2218 	ls[-1]: 2597*xq + 3036 	lt[-1] 2539*xq^2 + 622*xq + 1206
lr[-1]: 0 	ls[-1]: 4417*xq^2 + 529*xq + 1993 	lt[-1] 3514*xq^3 + 4259*xq^2 + 49*xq + 1265

lr: [1466*xq^3 + 1816*xq^2 + 1502*xq + 3922, 4303*xq^2 + 2617*xq + 3932, 138*xq + 3916, 2218, 0] 
ls: [1, 0, 1, 2597*xq + 3036, 4417*xq^2 + 529*xq + 1993] 
lt: [0, 1, 2460*xq + 1254, 2539*xq^2 + 622*xq + 1206, 3514*xq^3 + 4259*xq^2 + 49*xq + 1265] 



(1, 3195*xq + 2982, 4499*xq^2 + 2480*xq + 3598)

In [56]:
jumpgcd_list( ff , gg )

jumpdivsteps_list( 5 1 [1466, 1816, 1502, 3922] [4303, 2617, 3932, 0] )
j= 2
jumpdivstep_mat( 2 1 [1466, 1816, 1502, 3922] [4303, 2617, 3932, 0] )
truncated( 2 ): f= [1466, 1816]   g= [4303, 2617]
j= 1
jumpdivstep_mat( 1 1 [1466, 1816] [4303, 2617] )
truncated( 1 ): f= [1466]   g= [4303]
P1:
[[[0], [1]], [[4303], [3125]]]
P1x(f,g): f= [4303, 2617]   g= [1920, 0]
jumpdivstep_mat( 1 0 [4303, 2617] [1920, 0] )
truncated( 1 ): f= [4303]   g= [1920]
P2:
[[[1], [0]], [[2671], [4303]]]
ret:  1 [[[0, 0], [0, 1]], [[306, 0], [4427, 2671]]]
P1:
[[[0, 0], [0, 1]], [[306, 0], [4427, 2671]]]
P1x(f,g): f= [4303, 2617, 3932, 0]   g= [909, 45, 0, 0]
jumpdivstep_mat( 3 1 [4303, 2617, 3932, 0] [909, 45, 0, 0] )
truncated( 3 ): f= [4303, 2617, 3932]   g= [909, 45, 0]
j= 1
jumpdivstep_mat( 1 1 [4303, 2617, 3932] [909, 45, 0] )
truncated( 1 ): f= [4303]   g= [909]
P1:
[[[0], [1]], [[909], [288]]]
P1x(f,g): f= [909, 45, 0]   g= [4493, 2390, 0]
jumpdivstep_mat( 2 0 [909, 45, 0] [4493, 2390, 0] )
truncated( 2

(1, [1, 0, 0, 0], [3195, 2982], [4499, 2480, 3598])

In [22]:
a16 = random_poly(16)
b15 = random_poly(15)

In [30]:
jumpgcd_list(a16,b15)

jumpdivsteps_list( 31 1 [2446, 1172, 1794, 3526, 1312, 2300, 1682, 1618, 1396, 3553, 4525, 718, 1404, 1932, 1213, 2753, 1393] [2351, 1482, 2194, 1696, 46, 3572, 3847, 1716, 4020, 207, 350, 1090, 102, 634, 4559, 2712, 0] )
j= 15
jumpdivstep_mat( 15 1 [2446, 1172, 1794, 3526, 1312, 2300, 1682, 1618, 1396, 3553, 4525, 718, 1404, 1932, 1213, 2753, 1393] [2351, 1482, 2194, 1696, 46, 3572, 3847, 1716, 4020, 207, 350, 1090, 102, 634, 4559, 2712, 0] )
truncated( 15 ): f= [2446, 1172, 1794, 3526, 1312, 2300, 1682, 1618, 1396, 3553, 4525, 718, 1404, 1932, 1213]   g= [2351, 1482, 2194, 1696, 46, 3572, 3847, 1716, 4020, 207, 350, 1090, 102, 634, 4559]
j= 7
jumpdivstep_mat( 7 1 [2446, 1172, 1794, 3526, 1312, 2300, 1682, 1618, 1396, 3553, 4525, 718, 1404, 1932, 1213] [2351, 1482, 2194, 1696, 46, 3572, 3847, 1716, 4020, 207, 350, 1090, 102, 634, 4559] )
truncated( 7 ): f= [2446, 1172, 1794, 3526, 1312, 2300, 1682]   g= [2351, 1482, 2194, 1696, 46, 3572, 3847]
j= 3
jumpdivstep_mat( 3 1 [2446, 1172, 17

(1,
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [3830,
  2050,
  1936,
  1171,
  620,
  3886,
  492,
  4146,
  186,
  828,
  2358,
  2396,
  1812,
  256,
  400,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [616,
  1838,
  1810,
  379,
  4364,
  2488,
  3948,
  3010,
  1147,
  3206,
  2360,
  3049,
  2434,
  425,
  70,
  988,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [31]:
ext_gcd(a16,b15)

lr[0]:  2446*xq^16 + 1172*xq^15 + 1794*xq^14 + 3526*xq^13 + 1312*xq^12 + 2300*xq^11 + 1682*xq^10 + 1618*xq^9 + 1396*xq^8 + 3553*xq^7 + 4525*xq^6 + 718*xq^5 + 1404*xq^4 + 1932*xq^3 + 1213*xq^2 + 2753*xq + 1393 	ls[0]: 1 	lt[0]: 0
lr[1]:  2351*xq^15 + 1482*xq^14 + 2194*xq^13 + 1696*xq^12 + 46*xq^11 + 3572*xq^10 + 3847*xq^9 + 1716*xq^8 + 4020*xq^7 + 207*xq^6 + 350*xq^5 + 1090*xq^4 + 102*xq^3 + 634*xq^2 + 4559*xq + 2712 	ls[1]: 0 	lt[0]: 1

lr[-1]: 941*xq^14 + 278*xq^13 + 704*xq^12 + 3802*xq^11 + 434*xq^10 + 1708*xq^9 + 2918*xq^8 + 2507*xq^7 + 4001*xq^6 + 360*xq^5 + 4044*xq^4 + 4433*xq^3 + 4239*xq^2 + 2973*xq + 2595 	ls[-1]: 1 	lt[-1] 3637*xq + 58
lr[-1]: 2016*xq^13 + 4569*xq^12 + 2239*xq^11 + 3910*xq^10 + 605*xq^9 + 4179*xq^8 + 954*xq^7 + 1782*xq^6 + 924*xq^5 + 1376*xq^4 + 2332*xq^3 + 4489*xq^2 + 3993*xq + 496 	ls[-1]: 3320*xq + 2062 	lt[-1] 510*xq^2 + 2129*xq + 231
lr[-1]: 2574*xq^12 + 17*xq^11 + 4220*xq^10 + 4395*xq^9 + 4590*xq^8 + 727*xq^7 + 3194*xq^6 + 3286*xq^5 + 1808*xq^4 + 3479*xq^

(1,
 3830*xq^14 + 2050*xq^13 + 1936*xq^12 + 1171*xq^11 + 620*xq^10 + 3886*xq^9 + 492*xq^8 + 4146*xq^7 + 186*xq^6 + 828*xq^5 + 2358*xq^4 + 2396*xq^3 + 1812*xq^2 + 256*xq + 400,
 616*xq^15 + 1838*xq^14 + 1810*xq^13 + 379*xq^12 + 4364*xq^11 + 2488*xq^10 + 3948*xq^9 + 3010*xq^8 + 1147*xq^7 + 3206*xq^6 + 2360*xq^5 + 3049*xq^4 + 2434*xq^3 + 425*xq^2 + 70*xq + 988)